# Image segmentation

Since the primary aim of the `Band_Data` and `Data` classes we have recently been looking at is to produce a [Catalogue](../catalogue.rst) of sources, we will also need to be able to identify one object from another. This is done in a process known as segmentation, and in principle is quite challenging to do well, although we will have a go here.

# Example 1: Making segmentation maps with SExtractor

The first technique we will use to segment the data into different sources, which is the default in galfind, is to use the SExtractor parameters from Adams et al. 2023. Before starting this example please ensure that this is installed appropriately on your computer; an explanation of how to install SExtractor can be found [here](../getting_started/installation.rst). Let's start by instantiating our JOF data object as before.

In [ ]:
from copy import deepcopy
import matplotlib.pyplot as plt
from galfind import config, Data
from galfind.Data import morgan_version_to_dir

survey = "JOF"
version = "v11"
instrument_names = ["NIRCam"]

JOF_data = Data.from_survey_version(
    survey = survey,
    version = version,
    instrument_names = instrument_names, 
    version_to_dir_dict = morgan_version_to_dir,
)

To look at what exactly the segmentation process is doing, we will try it on the F444W band only. This will take a little while as SExtractor is running; the output will be stored in the config `SExtractor/SEX_DIR` directory when it has finished under the appropriate survey/version/instrument sub-directories (which in this case this is JOF/v11/NIRCam).

In [ ]:
F444W_JOF = JOF_data["F444W"]
F444W_JOF_2 = deepcopy(F444W_JOF)
F444W_JOF_3 = deepcopy(F444W_JOF)
F444W_JOF_4 = deepcopy(F444W_JOF)

F444W_JOF.segment()
print(F444W_JOF)

In addition to the segmentation map, this function will additionally produce a background map and photometric catalogue with fluxes (from sources selected in the same band) in apertures set by the `SExtractor/APER_DIAMS` parameter in the config file. The `seg_path` and the method used to measure this will be saved inside the `Band_Data` class for future access. 

Now let's say we wanted to load the path to the segmentation map rather than making it. We can simply run this function again on a fresh `Band_Data` object, and if the required file already exists it will load the path to it.

In [ ]:
F444W_JOF_2.segment()

We see from above that SExtractor is not called here. If we wanted to re-run this segmentation map for whatever reason, there are two possible actions we could take:

1. Delete or move the saved segmentation map manually
2. Pass `overwrite=True` into the `segment` function on a pre-segmented object

Option 2 is the preferred, and probably more straight forward, choice to make here. Below is its implementation.

In [ ]:
# re-segment using SExtractor
F444W_JOF_3.segment(overwrite=True)

Excellent! We have overwritten the original segmentation map, albeit with the exact same SExtractor output. Next we will produce a segmentation map using the stored `wht` map rather than the `rms_err` map by changing the `err_type` attribute from the default `rms_err` to `wht`. 

In [ ]:
F444W_JOF_4.segment("wht", overwrite=True)
print(F444W_JOF_4)

This will update the segmentation map path in the `Band_Data` object for F444W and make a new segmentation map using the `wht` path, although will not actually overwrite the previously run `rms_err` map. The difference between the two maps can be seen by comparing the segmentation maps generated from the two separate weight maps.

In [ ]:
# load the segmentation maps
rms_err_seg = F444W_JOF.load_seg()
wht_seg = F444W_JOF_4.load_seg()
# plot the difference
fig, ax = plt.subplots()
ax.imshow(rms_err_seg - wht_seg)
plt.show()

Lastly, we will regularly want to produce segmentation maps for every band used in a particular survey, for instance when running a cataloguing pipeline. As before, this takes in `err_type`, `method`, and `overwrite` arguments; we will implement the default functionality of this below.

In [ ]:
JOF_data.segment(err_type="rms_err", method="sextractor", overwrite=False)
print(JOF_data)

## Example 2: Plotting the segmentation map

Once the segmentation map has been created, the next step is to plot it. We use the `Data.plot()` function using the `ext = "SEG"` argument. There are a few more arguments that we write explicitly for clarity, although we note that for now we will keep these as the galfind defaults.

In [ ]:
JOF_data.plot(ext = "SEG")

Maybe we also want to zoom into a specific region of the segmentation map as well to have a closer look at some specific sources. Let's change some of the default arguments now to see how this changes the output plot.